In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import cv2
from typing import List,Dict,Optional
import torch

In [2]:
root=Path('codotoliths')
otolith_dirs=list(root.rglob("Nr*_age*"))

In [3]:
print(f"Nombre d'otholites : {len(otolith_dirs)}")

Nombre d'otholites : 1029


#### Parmi les 1029 dossiers, 28 dossiers contiennent plus de 6 images. En se basant sur notre dernière réunion avec notre tutrice, on va pas travailler avec ces dossiers. Chaque dossier conservé contient 3 images d'exposition 0 deg, les 3 autres 180 deg. On va travailler avec une seule image par exposition
#### Donc le résultat du cleaning sera 1021 dossiers chacun contenant 2 images d'otolithe: une 0 deg, l'autre 180 deg
#### On va travailler avec les images à leur résolution d'origine et non pas ce fusionner. En nous appuyant sur ces 2 papiers : 

#### https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0204713&amp=&type=printable
#### https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0204713     

#### Les CNN donnent de meilleurs résultats avec des images brutes, et la fusion risque d'entrainer une perte d'informatoins.
#### Et la derniere fois on a utiliser CLAHE pour augmenterr la visualisation des anneaux de l'otolithe, bien que cette méthode améliore la visualisation des anneaux pour l'oeuil humain, CLAHE ppeuvent introsuire des artefacts, et créer des patterns qui n'existent pas.
#### Ce qu'on peut faire c'est qu'on travaille avec les deux methodes, les 3 approches on va les traiter avec les 2 dossiers (AVEC CLAHE, SANS CLAHE) et on compare les résultats des deux. Afin de savoir est ce que c'est mieux de travailler avec les images brutes, ou bien les imges traitées.

## Pré-traitement des images -SANS CLAHE- 

### Les dossiers qui contiennent exactement 6 images seront placés dans un dossier, et ceux qui contiennent plus ou moins de 6 seront placés dans un autre dossier : 

In [21]:
dossier_cplt=[]
dossier_pcplt=[]
for otolith_dir in otolith_dirs:
    image_paths = sorted(otolith_dir.glob("*.jpg"))
    if len(image_paths)!=6:
        dossier_pcplt.append(otolith_dir)
    else :
        dossier_cplt.append(otolith_dir)
        
print(f"Le nombre total des dossiers qui suivent la forme est : {len(dossier_cplt)}")
print(f"Le nombre total des dossiers qui ne suivent pas la forme est : {len(dossier_pcplt)}")

Le nombre total des dossiers qui suivent la forme est : 1001
Le nombre total des dossiers qui ne suivent pas la forme est : 28


#### On selectionne les images 0 deg et 180 deg avec résolution normale

In [22]:
def img_0deg_180deg(folder):
    images = sorted([f for f in folder.glob('*.jpg')])
    image_0deg=images[0]
    image_180deg=images[3]
    return image_0deg,image_180deg

In [23]:
def redimensionnement(input_path,output_path,size):
    img=cv2.imread(str(input_path))
    if img is None:
        raise ValueError(f"L'image {path} ne se charge pas")
    img_resized = cv2.resize(img,size, interpolation=cv2.INTER_LANCZOS4)
    if len(img_resized.shape)==2:
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    output_path.parent.mkdir(parents=True,exist_ok=True)
    cv2.imwrite(str(output_path), img_resized)
    return True

#### Pour la première approche : Approche directe : Image brute → Deep Learning → Âge
##### Cette approche est basée sur Convolutional neural network (CNN) afin de prédire l'age du poisson à partir de l'otolithe.
##### Il existe différentes methodes qu'on peut utiliser : 
##### - InceptionV3, Xception (299x299) 
##### - ResNet, VGG, MobileNet (224x224) 
##### - EfficientNet, custom high-res (512x512) 
##### C'est pour cela qu'on va avoir plusieurs dossiers pour la première approche et chaque dossier correspond à l'une des méthodes.

##### N.B: Les images doivent etre normalisées pendant l'entrainement, et étant donné que maintenant on est uniquement en phase de prétraitement, on va uniquement les redimensionner. Les images ont originellement une résolution de 5616 x 3744 pixels, ce qui est une très grandes résolutions.

In [24]:
output_path=Path('premiere_approche')

In [25]:
methods={
    'InceptionV3':(299,299),
    'ResNet':(224,224),
    'EfficientNet':(512,512)
}

In [26]:
for method_name,method_size in methods.items():
    for otlt in tqdm(dossier_cplt):
        image_0deg,image_180deg=img_0deg_180deg(folder)
        folder_name=otlt.name
        output_folder=output_path/method_name/folder_name
        output_folder.parent.mkdir(parents=True,exist_ok=True)
        redimensionnement(image_0deg,output_folder/"image_0deg.png",method_size)
        redimensionnement(image_180deg,output_folder/"image_180deg.png",method_size)

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [09:03<00:00,  1.84it/s]


#### Pour la deuxième approche : Approche intermédiaire :  Image brute -> segmentation par DL -> comptage strie.
##### Cette approche repose sur la segmentation des otolithes par deep learning afin d'isoler et de compter les stries de croissance, qui permettent d'estimer l'age du poisson. 
##### Les méthodes adaptées à la segmentation sont :
##### - U-Net, DeepLab, FCN (512x512) 
##### - U-Net rapide (lightweight) (256x256) 
##### - Mask R-CNN (1024x1024)

##### C'est pour cela qu'on va avoir plusieurs dossiers pour la deuxième approche et chaque dossier correspond à l'une des méthodes.

##### N.B: Les images doivent etre normalisees pendant lèentrainement, et puisque maintenant on est juste en prétraitement, on va juste redimensionner les images, de base ils sont de taille 5616x3744 pixels ce qui est une tres grandes resolutions.

In [27]:
methods_approche2={
    'unet_deeplab_fcn':(512,512),
    'unet_rapid':(256,256),
    'RCNN':(1024,1024)
}

In [28]:
output_path_deuxieme=Path('deuxieme_approche')

In [29]:
for method_name,method_size in methods_approche2.items():
    for otlt in tqdm(dossier_cplt):
        image_0deg,image_180deg=img_0deg_180deg(folder)
        folder_name=otlt.name
        output_folder=output_path_deuxieme/method_name/folder_name
        output_folder.parent.mkdir(parents=True,exist_ok=True)
        redimensionnement(image_0deg,output_folder/"image_0deg.png",method_size)
        redimensionnement(image_180deg,output_folder/"image_180deg.png",method_size)

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [11:19<00:00,  1.47it/s]


#### Pour la troisième approche : Approche hybride : Image → Transformée Mojette → Deep Learning → Âge + Stries
##### Pour cette approche, il faut d'abord binariser les images puisqu'on va travailler avec la transformée Mojette
##### On va travailler avec 3 méthodes de binarisation: 
##### - Binarisation OTSU
##### - Seuillage adaptif
##### - OTSU + Nettoyage morphologique
##### Chaque méthode de binarisation correspondra à un dossier distinct au sein de cette troisième approche.

In [30]:
def binarisation_otsu(input_path,output_path):
    img = cv2.imread(str(input_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"L'image {path} ne se charge pas")
    img_rsd = cv2.resize(img,(256,256), interpolation=cv2.INTER_LANCZOS4)
    _, img_binary = cv2.threshold(img_rsd, 0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), img_binary)
    return True

In [31]:
def binarisation_adaptive(input_path,output_path):
    img = cv2.imread(str(input_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"L'image {path} ne se charge pas")
    img_rsd = cv2.resize(img,(256,256), interpolation=cv2.INTER_LANCZOS4)
    img_binary = cv2.adaptiveThreshold(img_rsd,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), img_binary)
    return True

In [32]:
def binarisation_OTSU_NTG(input_path,output_path):
    img = cv2.imread(str(input_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"L'image {path} ne se charge pas")
    img_rsd = cv2.resize(img,(256,256), interpolation=cv2.INTER_LANCZOS4)
    _, img_binary = cv2.threshold(img_rsd, 0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    img_binary = cv2.morphologyEx(img_binary, cv2.MORPH_CLOSE, kernel)
    img_binary = cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, kernel)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), img_binary)
    return True

In [36]:
output_path_troisieme=Path('troisieme_approche')

In [37]:
for otlt in tqdm(dossier_cplt):
    image_0deg,image_180deg=img_0deg_180deg(folder)
    folder_name=otlt.name
    output_folder=output_path_troisieme/'binarisation_otsu'/folder_name
    output_folder.parent.mkdir(parents=True,exist_ok=True)
    binarisation_otsu(image_0deg,output_folder/"image_0deg.png")
    binarisation_otsu(image_180deg,output_folder/"image_180deg.png")

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [04:09<00:00,  4.02it/s]


In [38]:
for otlt in tqdm(dossier_cplt):
    image_0deg,image_180deg=img_0deg_180deg(folder)
    folder_name=otlt.name
    output_folder=output_path_troisieme/'binarisation_adaptive'/folder_name
    output_folder.parent.mkdir(parents=True,exist_ok=True)
    binarisation_adaptive(image_0deg,output_folder/"image_0deg.png")
    binarisation_adaptive(image_180deg,output_folder/"image_180deg.png")

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [04:11<00:00,  3.98it/s]


In [39]:
for otlt in tqdm(dossier_cplt):
    image_0deg,image_180deg=img_0deg_180deg(folder)
    folder_name=otlt.name
    output_folder=output_path_troisieme/'binarisation_OTSU_NTG'/folder_name
    output_folder.parent.mkdir(parents=True,exist_ok=True)
    binarisation_OTSU_NTG(image_0deg,output_folder/"image_0deg.png")
    binarisation_OTSU_NTG(image_180deg,output_folder/"image_180deg.png")

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [04:16<00:00,  3.91it/s]
